<div align="center">

# Prédiction de la souscription d'un dépôt à terme  

## Analyse exploratoire des données (EDA)

<img src="https://raw.githubusercontent.com/komiadok/termdepositforecast/main/cover_image.jpg" width="500"/>

</div>

<div style="background-color:#008080; color:white; padding:15px; border-radius:8px; font-weight:bold; font-size:16px;">
📚 Chargement des librairies
</div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

<div style="background-color:#008080; color:white; padding:15px; border-radius:8px; font-weight:bold; font-size:16px;">
🗃️ Partie 1 : Importation des données
</div>

In [2]:
# Chargement du dataset
data = pd.read_csv('../data/raw/bank-additional-full.csv', sep = ';')
print(f'Taille du dataframe : {data.shape}')

# Aperçu du dataset
data.head()

Taille du dataframe : (41188, 21)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


<div style="background-color:#008080; color:white; padding:15px; border-radius:8px; font-weight:bold; font-size:16px;">
⚙️ Partie 2 : Data Preprocessing
</div>

In [3]:
# Définition du chemin où se trouve utils
import sys
sys.path.append('../pipelines/')  

# Importation de la classe
from preprocess import DataCleaningPipeline

In [4]:
# Instancier le pipeline
pipeline = DataCleaningPipeline(
    min_age=18,
    max_duration=300,
    max_calls=5,
    output_dir='../data/outputs'
)

# Transformation du DataFrame
cleaned_data = pipeline.fit_transform(data)
print(f'Taille du dataframe : {cleaned_data.shape}')

# Vérifier les premières lignes
cleaned_data.head()

Taille du dataframe : (41183, 31)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,missing_job,missing_marital,missing_education,missing_default,missing_housing,missing_loan,outlier_duration,outlier_campaign,outlier_previous,never_contacted
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,False,False,False,False,False,False,False,False,False,True
1,57,services,married,high.school,missing,no,no,telephone,may,mon,...,False,False,False,True,False,False,False,False,False,True
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,False,False,False,False,False,False,False,False,False,True
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,False,False,False,False,False,False,False,False,False,True
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,False,False,False,False,False,False,True,False,False,True
